In [ ]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import gym
import numpy as np
import keras
from keras import ops
from keras import layers
import tensorflow as tf

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/util.py:55: DeprecationWarning: jax.xla_computation is deprecated. Please use the AOT APIs; see https://jax.readthedocs.io/en/latest/aot.html. For example, replace xla_computation(f)(*xs) with jit(f).lower(*xs).compiler_ir('hlo'). See CHANGELOG.md for 0.4.30 for more examples.
  from jax import xla_computation as _xla_computation


In [ ]:
# Environment setup
env = gym.make("CartPole-v1")
num_inputs = 4
num_actions = 1  # Deterministic actions
num_hidden = 128
gamma = 0.99
exploration_noise_std = 0.1
max_steps_per_episode = 500

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [ ]:
# Actor model
actor_inputs = layers.Input(shape=(num_inputs,))
actor_hidden = layers.Dense(num_hidden, activation="relu")(actor_inputs)
actor_output = layers.Dense(num_actions, activation="tanh")(actor_hidden)
actor_model = tf.keras.Model(inputs=actor_inputs, outputs=actor_output)

In [ ]:
# Critic model
critic_inputs = layers.Input(shape=(num_inputs + num_actions,))
critic_hidden = layers.Dense(num_hidden, activation="relu")(critic_inputs)
q_value = layers.Dense(1)(critic_hidden)
critic_model = tf.keras.Model(inputs=critic_inputs, outputs=q_value)

actor_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
critic_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [ ]:
# Optimizers
actor_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
critic_optimizer = tf.keras.optimizers.Adam(learning_rate=0.002)

# Replay buffer setup
class ReplayBuffer:
    def __init__(self, buffer_capacity=100000, batch_size=64):
        self.buffer_capacity = buffer_capacity
        self.batch_size = batch_size
        self.buffer_counter = 0
        self.state_buffer = np.zeros((buffer_capacity, num_inputs))
        self.action_buffer = np.zeros((buffer_capacity, num_actions))
        self.reward_buffer = np.zeros((buffer_capacity, 1))
        self.next_state_buffer = np.zeros((buffer_capacity, num_inputs))
        self.done_buffer = np.zeros((buffer_capacity, 1))

    def store(self, state, action, reward, next_state, done):
        index = self.buffer_counter % self.buffer_capacity
        self.state_buffer[index] = state
        self.action_buffer[index] = action
        self.reward_buffer[index] = reward
        self.next_state_buffer[index] = next_state
        self.done_buffer[index] = done
        self.buffer_counter += 1

    def sample(self):
        max_buffer = min(self.buffer_counter, self.buffer_capacity)
        batch_indices = np.random.choice(max_buffer, self.batch_size)
        return (
            self.state_buffer[batch_indices],
            self.action_buffer[batch_indices],
            self.reward_buffer[batch_indices],
            self.next_state_buffer[batch_indices],
            self.done_buffer[batch_indices],
        )

# Initialize replay buffer
buffer = ReplayBuffer()

In [ ]:
# Training loop
for episode in range(1000):
    state = env.reset()
    episode_reward = 0

    for step in range(max_steps_per_episode):
        state_tensor = tf.convert_to_tensor(state, dtype=tf.float32)
        state_tensor = tf.expand_dims(state_tensor, axis=0)

        # Actor chooses action deterministically
        action = actor_model(state_tensor).numpy()[0]
        action += np.random.normal(0, exploration_noise_std)  # Add noise for exploration
        action = np.clip(action, -1, 1)  # Ensure action is within valid range

        # Execute action in environment
        next_state, reward, done, _ = env.step(int(action > 0))  # Convert continuous action to discrete
        episode_reward += reward

        # Store transition in replay buffer
        buffer.store(state, action, reward, next_state, done)

        # Train models if the replay buffer has enough samples
        if buffer.buffer_counter >= buffer.batch_size:
            states, actions, rewards, next_states, dones = buffer.sample()

            # Convert to tensors
            states = tf.convert_to_tensor(states, dtype=tf.float32)
            actions = tf.convert_to_tensor(actions, dtype=tf.float32)
            rewards = tf.convert_to_tensor(rewards, dtype=tf.float32)
            next_states = tf.convert_to_tensor(next_states, dtype=tf.float32)
            dones = tf.convert_to_tensor(dones, dtype=tf.float32)

            # Critic training
            with tf.GradientTape() as tape:
                # Target Q-value
                target_actions = actor_model(next_states)
                target_q_values = rewards + (1 - dones) * gamma * critic_model(
                    tf.concat([next_states, target_actions], axis=1)
                )
                # Predicted Q-value
                predicted_q_values = critic_model(tf.concat([states, actions], axis=1))
                critic_loss = tf.keras.losses.MSE(target_q_values, predicted_q_values)

            critic_grads = tape.gradient(critic_loss, critic_model.trainable_variables)
            critic_optimizer.apply_gradients(zip(critic_grads, critic_model.trainable_variables))

            # Actor training
            with tf.GradientTape() as tape:
                # Actor loss is the negative of the Q-value
                actions_pred = actor_model(states)
                actor_loss = -tf.reduce_mean(critic_model(tf.concat([states, actions_pred], axis=1)))

            actor_grads = tape.gradient(actor_loss, actor_model.trainable_variables)
            actor_optimizer.apply_gradients(zip(actor_grads, actor_model.trainable_variables))

        if done:
            break
        state = next_state

    print(f"Episode {episode + 1}: Reward: {episode_reward:.2f}")

    if episode_reward > 475:
        print("Solved!")
        break

<ipython-input-6-429cab3f941e>:16: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  next_state, reward, done, _ = env.step(int(action > 0))  # Convert continuous action to discrete
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode 1: Reward: 25.00
Episode 2: Reward: 11.00
Episode 3: Reward: 13.00
